In [10]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer


In [11]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer


In [12]:
train_file = 'blp25_hatespeech_subtask_1A_train.tsv'
validation_file = 'blp25_hatespeech_subtask_1A_dev.tsv'
test_file = 'blp25_hatespeech_subtask_1A_dev_test.tsv'

#tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
# Load train/val/test DataFrames
train_df = pd.read_csv(train_file, sep="\t")
dev_df = pd.read_csv(validation_file , sep="\t")
test_df = pd.read_csv(test_file, sep="\t")






In [13]:
pip install openai==0.28

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import openai

# Set up your OpenAI API key
openai.api_key = ""

# Function to prompt GPT-4 turbo
def query_model(query):
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",  # You can specify 'gpt-4', 'gpt-3.5-turbo', etc.
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": query}
        ],
        max_tokens=150,  # Control the length of the response
        temperature=0.7,  # Adjust for more deterministic or creative outputs
    )
    return response['choices'][0]['message']['content']

# Example usage
query = "Explain the concept of entropy in machine learning."
response = query_model(query)
print(response)


Entropy is a fundamental concept in information theory that measures the uncertainty or randomness in a set of data. In the context of machine learning, particularly in classification tasks, entropy is used to quantify the impurity or disorder of a dataset, helping to make decisions about how to split data in algorithms like decision trees.

### Key Points about Entropy:

1. **Definition**:
   - Entropy, denoted as \( H(S) \), for a discrete random variable is defined mathematically as:
     \[
     H(S) = -\sum_{i=1}^{n} p_i \log_2(p_i)
     \]
     where \( p_i \) is the probability of class \( i \) in the dataset \( S


In [15]:
import os
import openai
import pandas as pd
from tqdm import tqdm
import time


In [25]:
SYSTEM_PROMPT = "You are an expert Bangla text classification assistant. You are given a Bangla comment and your task is to classify it for toxicity."

def build_prompt(bangla_comment: str) -> str:
    return f"""
Instruction:
You are given a comment in Bangla in the context Bangladesh. Your task is to classify it for toxicity.  


Here are some examples:

Input: ইন্ডিয়া কি মাছ ধরা বন্ধ রাখছেএক নদীতে দুইনীতি কেমনে হয়  
Output: "Political Hate"

Input: লক্ষ টাকা ঘুষ দিয়ে অযোগ্য আর দায়িত্বহীন মানসিকতার মানুষ গুলো সরকারি চাকরিতে কাজ করেন আর সেই দায় এভাবেই দিতে হচ্ছে হাজারো মানুষের জীবন দিয়ে  
Output: "Abusive"

Input: ওহা ভবনের দালাল  
Output: "None"

Input: শালার ব্যাটা খুব বকা দিতে পারে ইচ্ছা করে না নির্লজ্জ কোথাকার ক্যামেরা সামনে কথা বলতে লজ্জা বোধ করে না  
Output: "Profane"

Input: খানকি মাগির মুখে মনে হয় ধন ছোয়া দেই  
Output: "Sexism"

Input: তারা হলো কাফেরতাদের জন্য জাহান্নাম নিদারুণ করছেন আললাহ  
Output: "Religious Hate"

Now classify the following comment:

Input: {bangla_comment}

Output:
One of: "None", "Religious Hate", "Sexism", "Political Hate", "Profane", "Abusive".
""".strip()


In [26]:
def query_model(user_input: str) -> str:
    prompt = build_prompt(user_input)
    # Simple retry for transient errors / rate limits
    for attempt in range(3):
        try:
            resp = openai.ChatCompletion.create(
                model="gpt-4o-mini",  # valid chat model name
                #model="gpt-4o",
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": prompt}
                ],
                temperature=0,         # deterministic translations
                max_tokens=300
            )
            return resp["choices"][0]["message"]["content"].strip()
        except openai.error.RateLimitError:
            time.sleep(2**attempt)
        except openai.error.OpenAIError as e:
            # surface other API errors
            raise e
    raise RuntimeError("Failed after retries")

In [30]:
import pandas as pd

df = pd.read_csv("llm_bnlp - Sheet3.csv")


df.columns

Index(['id', 'text', 'Eng_Text', 'Unnamed: 3'], dtype='object')

In [31]:
import os
import openai
import pandas as pd
from tqdm import tqdm
import time




for i in tqdm(range(1, 2)):
    question_col = "text"
    answer_col = f"Answer_few{i}"
    if question_col in df.columns:
        df[answer_col] = df[question_col].astype(str).apply(
            lambda text: query_model(text)
        )
        


100%|██████████| 1/1 [19:49<00:00, 1189.52s/it]


In [32]:
df

,id,text,Eng_Text,Unnamed: 3,Answer_few1
0,879187,শুভ কামনা রইল বাংলাদেশ জন্য ইনশাআল্লাহ জয় হবে,"Best wishes for Bangladesh, inshallah victory","Best wishes for Bangladesh, inshallah victory","Output: ""None"""
1,316919,গোয়া মারা দিয়ে আছে বাংলাদেশ মাদারচোদ নিউজ করে ...,Goa has killed Bangladesh motherchod news Sala...,Goa has killed Bangladesh motherchod news Sala...,"Output: ""Profane"""
2,916242,ভাইয়া আপনি অভিনেতা হইয়েন না না হলে সবাই বাচ্...,Bro you don't become an actor or else everyone...,Bro you don't become an actor or else everyone...,"Output: ""None"""
3,786824,আমাদেরো তাই দেখছি,We see the same,We see the same,"Output: ""None"""
4,47284,পুলিশ কতটা টাকা নিয়ে,How much money does the police take?,How much money does the police take?,"Output: ""None"""
...,...,...,...,...,...
2507,776466,সত্য কথা তেতু লাগে,The truth is bitter,The truth is bitter,"Output: ""None"""
2508,849227,এই ফকিননি মাগীটা আর কত নাটক দেখাবে,How many more dramas will this fakinini magita...,How many more dramas will this fakinini magita...,"Output: ""Profane"""
2509,532697,দেখো আজকে কার ফিটনেস কোথায় দাঁড়িয়েছে তুমি চ...,"Look who's fitness stands today, you're gone t...","Look who's fitness stands today, you're gone t...","Output: ""None"""
2510,861411,ছোট ভাইটির পাসে থাকুন গেম ভিড়িও বানাই,Stay close to the younger brother and make a g...,Stay close to the younger brother and make a g...,"Output: ""None"""


In [33]:
df['Answer_few1'].unique()

array(['Output: "None"', 'Output: "Profane"', 'Output: "Religious Hate"',
       'Output: "Political Hate"', 'Output: "Abusive"',
       'Output: "Sexism"', 'Output: "Racist"', 'Output: "Violent Threat"'],
      dtype=object)

In [40]:
df.to_csv("GPT-4.csv")

In [35]:
df

,id,text,Eng_Text,Unnamed: 3,Answer_few1
0,879187,শুভ কামনা রইল বাংলাদেশ জন্য ইনশাআল্লাহ জয় হবে,"Best wishes for Bangladesh, inshallah victory","Best wishes for Bangladesh, inshallah victory","Output: ""None"""
1,316919,গোয়া মারা দিয়ে আছে বাংলাদেশ মাদারচোদ নিউজ করে ...,Goa has killed Bangladesh motherchod news Sala...,Goa has killed Bangladesh motherchod news Sala...,"Output: ""Profane"""
2,916242,ভাইয়া আপনি অভিনেতা হইয়েন না না হলে সবাই বাচ্...,Bro you don't become an actor or else everyone...,Bro you don't become an actor or else everyone...,"Output: ""None"""
3,786824,আমাদেরো তাই দেখছি,We see the same,We see the same,"Output: ""None"""
4,47284,পুলিশ কতটা টাকা নিয়ে,How much money does the police take?,How much money does the police take?,"Output: ""None"""
...,...,...,...,...,...
2507,776466,সত্য কথা তেতু লাগে,The truth is bitter,The truth is bitter,"Output: ""None"""
2508,849227,এই ফকিননি মাগীটা আর কত নাটক দেখাবে,How many more dramas will this fakinini magita...,How many more dramas will this fakinini magita...,"Output: ""Profane"""
2509,532697,দেখো আজকে কার ফিটনেস কোথায় দাঁড়িয়েছে তুমি চ...,"Look who's fitness stands today, you're gone t...","Look who's fitness stands today, you're gone t...","Output: ""None"""
2510,861411,ছোট ভাইটির পাসে থাকুন গেম ভিড়িও বানাই,Stay close to the younger brother and make a g...,Stay close to the younger brother and make a g...,"Output: ""None"""


In [37]:
df['Answer_few1']

0          Output: "None"
1       Output: "Profane"
2          Output: "None"
3          Output: "None"
4          Output: "None"
              ...        
2507       Output: "None"
2508    Output: "Profane"
2509       Output: "None"
2510       Output: "None"
2511       Output: "None"
Name: Answer_few1, Length: 2512, dtype: object

In [38]:
df["Answer_few1"] = (
    df["Answer_few1"]
    .str.replace('Output:\s*"', '', regex=True)  # remove Output: and opening quote
    .str.replace('"', '', regex=True)            # remove closing quote
    .str.strip()                                 # clean spaces
)


<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_1942486/2280670092.py:3: SyntaxWarning: invalid escape sequence '\s'
  .str.replace('Output:\s*"', '', regex=True)  # remove Output: and opening quote


In [39]:
df

,id,text,Eng_Text,Unnamed: 3,Answer_few1
0,879187,শুভ কামনা রইল বাংলাদেশ জন্য ইনশাআল্লাহ জয় হবে,"Best wishes for Bangladesh, inshallah victory","Best wishes for Bangladesh, inshallah victory",None
1,316919,গোয়া মারা দিয়ে আছে বাংলাদেশ মাদারচোদ নিউজ করে ...,Goa has killed Bangladesh motherchod news Sala...,Goa has killed Bangladesh motherchod news Sala...,Profane
2,916242,ভাইয়া আপনি অভিনেতা হইয়েন না না হলে সবাই বাচ্...,Bro you don't become an actor or else everyone...,Bro you don't become an actor or else everyone...,None
3,786824,আমাদেরো তাই দেখছি,We see the same,We see the same,None
4,47284,পুলিশ কতটা টাকা নিয়ে,How much money does the police take?,How much money does the police take?,None
...,...,...,...,...,...
2507,776466,সত্য কথা তেতু লাগে,The truth is bitter,The truth is bitter,None
2508,849227,এই ফকিননি মাগীটা আর কত নাটক দেখাবে,How many more dramas will this fakinini magita...,How many more dramas will this fakinini magita...,Profane
2509,532697,দেখো আজকে কার ফিটনেস কোথায় দাঁড়িয়েছে তুমি চ...,"Look who's fitness stands today, you're gone t...","Look who's fitness stands today, you're gone t...",None
2510,861411,ছোট ভাইটির পাসে থাকুন গেম ভিড়িও বানাই,Stay close to the younger brother and make a g...,Stay close to the younger brother and make a g...,None
